In [1]:
import sqlite3

# connect to database
db_filename = 'squirrel_data.db'
conn = sqlite3.connect(db_filename)
c = conn.cursor()

In [2]:
from prettytable import PrettyTable

# utility function to query the database and display results
def table_from_query(cursor, query, table_columns, fetch_count=None):
    if fetch_count is None:
        f = cursor.execute(query).fetchall()
    else:
        f = cursor.execute(query).fetchmany(fetch_count)
    
    t = PrettyTable(table_columns)
    for row in f:
        t.add_row(row)
        
    print(t)

## 1. What was the date with the most squirrel sightings?

In [3]:
query = """SELECT 
    Date, 
    COUNT(*) AS Number_of_Sightings 
FROM squirrel_data
GROUP BY Date
ORDER BY Number_of_Sightings DESC
LIMIT 1;"""

columns = ('Date', 'Number of Sightings')

table_from_query(c, query, columns)

+----------+---------------------+
|   Date   | Number of Sightings |
+----------+---------------------+
| 10132018 |         434         |
+----------+---------------------+


October 13th 2018 had the most sightings. 434 squirrels were recorded on that date.

## 2. Which squirrels run more, adults or juveniles?

In [4]:
# create temporary table of number of running squirrels by age
q1 = "DROP TABLE IF EXISTS running_by_age;"
q2 = """CREATE TEMP TABLE running_by_age AS
SELECT 
    Age, 
    COUNT(*) AS Running_Count 
FROM squirrel_data
WHERE Running = true AND Age IN ("Adult", "Juvenile")
GROUP BY Age;"""

# create a temporary table of total number of squirrels by age
q3 = "DROP TABLE IF EXISTS total_by_age;"
q4 = """CREATE TEMP TABLE total_by_age AS
SELECT 
    Age, 
    COUNT(*) AS Total_Count 
FROM squirrel_data
WHERE Age IN ("Adult", "Juvenile")
GROUP BY Age;"""

for query in (q1, q2, q3, q4):
    c.execute(query)

# calculate percentage of squirrels running in each age group
q5 = """SELECT 
    t1.Age, 
    t1.Running_Count, 
    t2.Total_Count, 
    ROUND(100.0 * t1.Running_Count / t2.Total_Count, 1) AS Percentage_Running
FROM running_by_age AS t1
JOIN total_by_age AS t2
ON t1.Age = t2.Age;"""

columns = ('Age', 'Number of Running Squirrels', 'Total Squirrels', '% Running Squirrels')

table_from_query(c, q5, columns)

+----------+-----------------------------+-----------------+---------------------+
|   Age    | Number of Running Squirrels | Total Squirrels | % Running Squirrels |
+----------+-----------------------------+-----------------+---------------------+
|  Adult   |             625             |       2568      |         24.3        |
| Juvenile |              82             |       330       |         24.8        |
+----------+-----------------------------+-----------------+---------------------+


Adult squirrels and juvenile squirrels are roughly equally likely to be running. The "% Running Squirrels" column shows the percent of squirrels in each age category that were observed running. 24.3% of adult squirrels ran, compared to 24.8% of juveniles.

Most of the squirrels seen running were adults, as seen in the "Number of Running Squirrels" column. However, this does not mean that adult squirrels are more likely to run. Instead, it reflects that most recorded squirrel sightings were of adult squirrels (demonstrated by the "Total Squirrels" column).

## 3. In which areas are squirrels more willing to approach humans?

In [5]:
# create temporary table of squirrels aproaching by hectare
q1 = "DROP TABLE IF EXISTS approaching_by_hectare;"
q2 = """CREATE TEMP TABLE approaching_by_hectare AS
SELECT 
    Hectare, 
    COUNT(*) AS Approaching_Count 
FROM squirrel_data
WHERE Approaches = true
GROUP BY Hectare;"""

# create temporary table of total squirrels by hectare
q3 = "DROP TABLE IF EXISTS total_by_hectare;"
q4 = """CREATE TEMP TABLE total_by_hectare AS
SELECT 
    Hectare, 
    COUNT(*) AS Total_Count
FROM squirrel_data
GROUP BY Hectare;"""

for query in (q1, q2, q3, q4):
    c.execute(query)

# calculate percentage of squirrels approaching in each hectare
q5 = """SELECT 
    t1.Hectare, 
    t1.Approaching_Count, 
    t2.Total_Count, 
    ROUND(100.0 * t1.Approaching_Count / t2.Total_Count, 1) AS Percentage_Approaching
FROM approaching_by_hectare AS t1
JOIN total_by_hectare AS t2
ON t1.Hectare = t2.Hectare
ORDER BY Percentage_Approaching DESC
LIMIT 10;"""

columns = ('Hectare', 'Number of Approaching Squirrels', 'Total Squirrels', '% Approaching Squirrels')

table_from_query(c, q5, columns)

+---------+---------------------------------+-----------------+-------------------------+
| Hectare | Number of Approaching Squirrels | Total Squirrels | % Approaching Squirrels |
+---------+---------------------------------+-----------------+-------------------------+
|   23D   |                1                |        1        |          100.0          |
|   22E   |                1                |        2        |           50.0          |
|   34B   |                2                |        4        |           50.0          |
|   39I   |                2                |        4        |           50.0          |
|   03A   |                5                |        11       |           45.5          |
|   20C   |                4                |        9        |           44.4          |
|   17I   |                2                |        5        |           40.0          |
|   04B   |                4                |        11       |           36.4          |
|   05G   

The table above shows the 10 hectares with the greatest percentage of squirrels that approached humans (hectares 23D, 22E, 34B, 39I, 03A, 20C, 17I, 04B, 05G and 08A). However, because the sample sizes from these areas were very small (between 1 and 11 squirrels) the percentages of squirrels that approached may not reflect the likelihood of approaching humans in the full population.

## 4. Do squirrels that approach humans eat more frequently?

In [6]:
# create temporary table of number of squirrels seen eating, grouped by whether or not they approach humans
q1 = "DROP TABLE IF EXISTS eating_by_approaching"
q2 = """CREATE TEMP TABLE eating_by_approaching AS
SELECT 
    Approaches, 
    COUNT(*) AS Eating_Count 
FROM squirrel_data
WHERE Eating = true
GROUP BY Approaches;"""

# create temporary table of total number of squirrels seen, grouped by whether or not they approach humans 
q3 = "DROP TABLE IF EXISTS squirrels_by_approaching"
q4 = """CREATE TEMP TABLE squirrels_by_approaching AS
SELECT 
    Approaches, 
    COUNT(*) AS Squirrel_Count 
FROM squirrel_data
GROUP BY Approaches;"""

for query in (q1, q2, q3, q4):
    c.execute(query)
    
q5 = """SELECT
    t1.Approaches,
    t1.Eating_Count,
    t2.Squirrel_Count,
    ROUND(100.0 * t1.Eating_Count / t2.Squirrel_Count, 1) AS Percentage_Eating
FROM eating_by_approaching AS t1
JOIN squirrels_by_approaching AS t2
ON t1.Approaches = t2.Approaches"""

columns = ('Approaches', 'Number of Eating Squirrels', 'Total Squirrels', '% Eating Squirrels')

table_from_query(c, q5, columns)

+------------+----------------------------+-----------------+--------------------+
| Approaches | Number of Eating Squirrels | Total Squirrels | % Eating Squirrels |
+------------+----------------------------+-----------------+--------------------+
|     0      |            693             |       2845      |        24.4        |
|     1      |             67             |       178       |        37.6        |
+------------+----------------------------+-----------------+--------------------+


The squirrels that approached humans were about 13% more likely to be seen eating than the squirrels that did not approach.

## 5. Are squirrels that run away from humans more likely to quaa or moan?

The data includes columns of squirrel vocalizations that have different meanings. According to the data source, when a squirrel quaas it means they have seen a land predator. Moans indicate a squirrel has spotted an air predator.

In [7]:
query = """SELECT 
    COUNT(*) AS Running_Count, 
    SUM(Quaas) AS Quaa_Count, 
    SUM(Moans) AS Moan_Count 
FROM squirrel_data
WHERE [Runs from] = true"""

columns = ('Total Running Away', 'Quaa Count', 'Moan Count')

table_from_query(c, query, columns)

+--------------------+------------+------------+
| Total Running Away | Quaa Count | Moan Count |
+--------------------+------------+------------+
|        678         |     9      |     2      |
+--------------------+------------+------------+


Most squirrels that seen running from humans did not make either vocalization. The squirrels that vocalized were more likely to indicate that they had seen a land predator (by quaaing) than indicate they'd seen an air predator (by moaning).

## 6. Which areas have the most squirrels of each fur color

In [8]:
for color in ('Gray', 'Black', 'Cinnamon'):
    query = f"""SELECT 
        Hectare, 
        COUNT(*) AS {color}_Count 
    FROM squirrel_data
    WHERE [Primary Fur Color] LIKE "{color}"
    GROUP BY Hectare
    ORDER BY {color}_Count DESC
    LIMIT 3;"""
    
    columns = ('Hectare', f'{color} Squirrel Count')
    
    print(f'\nMost common locations of {color.lower()} squirrels:')
    table_from_query(c, query, columns)


Most common locations of gray squirrels:
+---------+---------------------+
| Hectare | Gray Squirrel Count |
+---------+---------------------+
|   14D   |          32         |
|   14E   |          28         |
|   32E   |          26         |
+---------+---------------------+

Most common locations of black squirrels:
+---------+----------------------+
| Hectare | Black Squirrel Count |
+---------+----------------------+
|   33D   |          8           |
|   33E   |          7           |
|   32E   |          4           |
+---------+----------------------+

Most common locations of cinnamon squirrels:
+---------+-------------------------+
| Hectare | Cinnamon Squirrel Count |
+---------+-------------------------+
|   04D   |            14           |
|   03D   |            12           |
|   05C   |            10           |
+---------+-------------------------+


Squirrels with different primary fur colors are most commonly found in different locations. Hectares 14D, 14E and 32E had the greatest number of gray squirrel sightings. Black squirrels were seen most often in hectares 33D, 33E and 32E. The hectares with the most cinnamon squirrel sightings were 04D, 03D and 05C.

## 7. How many squirrels engaged in each activity in the AM? In the PM?

In [9]:
query = """SELECT 
    Shift,
    SUM(Running) AS Running_Count,
    SUM(Chasing) AS Chasing_Count,
    SUM(Climbing) AS Climbing_Count,
    SUM(Eating) AS Eating_Count,
    SUM(Foraging) AS Foraging_Count
FROM squirrel_data
GROUP BY Shift;"""
columns = ('Shift', 'Running Count', 'Chasing Count', 'Climbing Count', 'Eating Count', 'Foraging Count')
table_from_query(c, query, columns)

+-------+---------------+---------------+----------------+--------------+----------------+
| Shift | Running Count | Chasing Count | Climbing Count | Eating Count | Foraging Count |
+-------+---------------+---------------+----------------+--------------+----------------+
|   AM  |      327      |      131      |      340       |     296      |      601       |
|   PM  |      403      |      148      |      318       |     464      |      834       |
+-------+---------------+---------------+----------------+--------------+----------------+


In both the AM and the PM, the most squirrel activity recorded was foraging. 601 squirrels were seen foraging in the AM and 834 foraged in the PM. The least common activity at both times of day was chasing (131 squirrels in the AM, 148 in the PM).

## 8. Are squirrels with certain fur colors more likely to approach humans?

In [10]:
q1 = "DROP TABLE IF EXISTS approaching_by_fur_color;"
q2 = """CREATE TEMP TABLE approaching_by_fur_color AS
SELECT 
    [Primary Fur Color], 
    COUNT(*) AS Approaching_Count
FROM squirrel_data
WHERE Approaches = true AND [Primary Fur Color] NOT NULL
GROUP BY [Primary Fur Color];"""

q3 = "DROP TABLE IF EXISTS squirrels_by_fur_color;"
q4 = """CREATE TEMP TABLE squirrels_by_fur_color AS
SELECT 
    [Primary Fur Color], 
    COUNT(*) AS Squirrel_Count
FROM squirrel_data
WHERE [Primary Fur Color] NOT NULL
GROUP BY [Primary Fur Color];"""

for query in (q1, q2, q3, q4):
    c.execute(query)
    
q5 = """SELECT
    t1.[Primary Fur Color],
    ROUND(100.0 * t1.Approaching_Count / t2.Squirrel_Count, 1) AS Percentage_Approaching
FROM approaching_by_fur_color AS t1
JOIN squirrels_by_fur_color AS t2
ON t1.[Primary Fur Color] = t2.[Primary Fur Color]
ORDER BY Percentage_Approaching DESC;"""
cols = ('Fur Color', '% Approaching')
table_from_query(c, q5, cols)

+-----------+---------------+
| Fur Color | % Approaching |
+-----------+---------------+
|  Cinnamon |      11.2     |
|   Black   |      5.8      |
|    Gray   |      5.1      |
+-----------+---------------+


Cinnamon-colored squirrels were most likely to approach humans. They were roughly twice as likely to approach as squirrels with other fur colors.

## 9. How many squirrels made more than one vocalization? What vocalizations did these squirrels make?

In [11]:
query = """WITH multiple_vocalizations AS (
    SELECT
        Kuks + Quaas + Moans AS Vocalization_Count,
        Kuks + Quaas = 2 AS Kuks_and_Quaas,
        Kuks + Moans = 2 AS Kuks_and_Moans,
        Quaas + Moans = 2 AS Quaas_and_Moans,
        Kuks + Quaas + Moans = 3 AS Kuks_Quaas_and_Moans
    FROM squirrel_data
)
SELECT
    COUNT(*) AS Multiple_Vocalizations_Count,
    SUM(Kuks_and_Quaas) AS Kuks_and_Quaas_Count,
    SUM(Kuks_and_Moans) AS Kuks_and_Moans_Count,
    SUM(Quaas_and_Moans) AS Quaas_and_Moans_Count,
    SUM(Kuks_Quaas_and_Moans) AS Kuks_Quaas_and_Moans_Count
FROM multiple_vocalizations
WHERE Vocalization_Count > 1;"""

cols = ('Multiple Vocalizations', 'Kuks and Quaas', 'Kuks and Moans', 
        'Quaas and Moans', 'All Vocalizations')

table_from_query(c, query, cols)

+------------------------+----------------+----------------+-----------------+-------------------+
| Multiple Vocalizations | Kuks and Quaas | Kuks and Moans | Quaas and Moans | All Vocalizations |
+------------------------+----------------+----------------+-----------------+-------------------+
|           16           |       15       |       0        |        1        |         0         |
+------------------------+----------------+----------------+-----------------+-------------------+


Squirrel vocalizations recorded in the data are kuks, quaas and moans. 16 squirrels made multiple vocalizations during a sighting. Of these squirrels, 15 kukked and quaaed, and 1 quaaed and moaned.

## 10. What are the mean and median number of squirrel sightings per day?

In [12]:
# get the number of sightings for each date
q1 = "DROP TABLE IF EXISTS sightings_by_date"
q2 = """CREATE TEMP TABLE sightings_by_date AS
SELECT 
    Date, 
    COUNT(*) AS Number_of_Sightings 
FROM squirrel_data
GROUP BY Date
ORDER BY Number_of_Sightings"""
c.execute(q1)
c.execute(q2)

# get the average sightings per day
q3 = """SELECT ROUND(AVG(Number_of_Sightings), 2) 
FROM sightings_by_date"""
mean = c.execute(q3).fetchone()[0]
print(f'Average daily squirrel sightings: {mean}')

# get the median sightings per day
# (SQLite doesn't have a median function, so we're doing this the hard way)
q4 = """SELECT ROUND(AVG(Number_of_Sightings), 2)
FROM (SELECT Number_of_Sightings
    FROM sightings_by_date
    LIMIT 2 - (SELECT COUNT(*) FROM sightings_by_date) % 2
    OFFSET (SELECT (COUNT(*) - 1) / 2 
        FROM sightings_by_date))"""
median = c.execute(q4).fetchone()[0]
print(f'Median daily squirrel sightings: {median}')

Average daily squirrel sightings: 274.82
Median daily squirrel sightings: 285.0


On the days that squirrel sightings were recorded, 274.82 squirrels were seen on average. The median number of daily squirrel sightings was 285. 